In [1]:
import re
import pandas as pd
import timeit
import nltk
import numpy as np
import string
from spell_check_dict import spelling_checker_dict
#stemmer = nltk.stem.porter.PorterStemmer()

In [133]:
#check you're in the right directory guys!
import os
os.getcwd()

'/Users/davida/Google Drive/IR & DM/IRDM Coursework'

In [134]:
import sys
sys.version

'3.5.2 |Anaconda custom (x86_64)| (default, Jul  2 2016, 17:52:12) \n[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]'

In [135]:

#YOU NEED TO DOWNLOAD THE STOPWORDS BELOW - BUT THIS MIGHT IMPROVE THE SCORE
stop_words=nltk.corpus.stopwords.words('english')
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

In [136]:
a=timeit.default_timer()
df_train = pd.read_csv('train.csv',encoding="ISO-8859-1")
df_test = pd.read_csv('test.csv',encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('product_descriptions.csv')
df_attr = pd.read_csv('attributes.csv')
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
len_train = df_train.shape[0]
b=timeit.default_timer()
print('loading and merging took {}s'.format(b-a))

loading and merging took 3.872367136005778s


In [137]:
#check what is actually missing
df_attr[df_attr.isnull().any(axis=1)]

#we see that for some attributes, values are missing, so these attributes become useless

before_len_df_attr=len(df_attr)
df_attr.dropna(inplace=True)
print('removed {} rows from df_attr'.format(before_len_df_attr-len(df_attr)))

removed 2090 rows from df_attr


In [138]:
from sklearn.utils import shuffle as sklearn_shuffle
a=timeit.default_timer()
df_train=sklearn_shuffle(df_train,random_state=2017)
b=timeit.default_timer()
print('shuffling train data took {}s'.format(b-a))

shuffling train data took 0.04189527401467785s


In [139]:
#PREPROCESSING PIPELINE
def unit_converter(s):
    try:
        #replace units
        s=re.sub(r"([0-9]+)( *)(inches|inch|in|in.|')\.?", r"\1 in. ",s)
        s=re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb|lb.)\.?", r"\1 lb. ",s)
        s=re.sub(r"([0-9]+)( *)(foot|feet|ft|ft.|'')\.?", r"\1 ft. ",s)
        s=re.sub(r"([0-9]+)( *)(square|sq|sq.) ?\.?(inches|inch|in|in.|')\.?", r"\1 sq.in. ",s)
        s=re.sub(r"([0-9]+)( *)(square|sq|sq.) ?\.?(feet|foot|ft|ft.|'')\.?", r"\1 sq.ft. ",s)
        s=re.sub(r"([0-9]+)( *)(cubic|cu|cu.) ?\.?(inches|inch|in|in.|')\.?", r"\1 cu.in. ", s)
        s=re.sub(r"([0-9]+)( *)(cubic|cu|cu.) ?\.?(feet|foot|ft|ft.|'')\.?", r"\1 cu.ft. ", s)
        s=re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1 gal. ",s)
        s=re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1 oz. ",s)
        s=re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1 cm. ",s)
        s=re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1 mm. ",s)
        s=re.sub(r"([0-9]+)( *)(minutes|minute)\.?", r"\1 min. ",s)
        s=re.sub(r"([0-9]+)( *)(°|degrees|degree)\.?", r"\1 deg. ",s)
        s=re.sub(r"([0-9]+)( *)(v|volts|volt)\.?", r"\1 volt. ",s)
        s=re.sub(r"([0-9]+)( *)(wattage|watts|watt)\.?", r"\1 watt. ",s)
        s=re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1 amp. ",s)
        s=re.sub(r"([0-9]+)( *)(qquart|quart)\.?", r"\1 qt. ",s)
        s=re.sub(r"([0-9]+)( *)(hours|hour|hrs.)\.?", r"\1 hr ",s)
        s=re.sub(r"([0-9]+)( *)(gallons per minute|gallon per minute|gal per minute|gallons/min.|gallons/min)\.?", \
         r"\1 gal. per min. ",s)
        s=re.sub(r"([0-9]+)( *)(gallons per hour|gallon per hour|gal per hour|gallons/hour|gallons/hr)\.?", r"\1 gal. per hr ",s)
        
        #replace things like '3x4 mm' or '3 by 4 mm' with '3 xbi 4'
        s = s.replace(" x "," xbi ")
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = s.replace('<b>','')
        s = s.replace('</b>','')
        s = s.replace('<br>','')
        s = s.replace('</br>','')
        #replace random weird stuff found after manual inspection
        s = s.replace('_',' ')
        s = s.replace('..','.')
        s = s.replace('\\',' ')
        s = s.replace('/',' ')
        s = s.replace('  ', ' ')
        s = s.replace('  ', ' ')
    except:
        raise TypeError(s,' is not a string')
    return s
def uppercase_splitter(s):
    #e.g. concrete surfaceActual to concrete surface Actual
    try:
        s=re.sub(r"(\w)[\.?!]([A-Z])", r"\1 \2",s)
        s=re.sub(r"(?<=( ))([a-z]+)([A-Z]+)", r"\2 \3",s)
        return s
    except:
        raise TypeError(s, 'is not a string')
def letter_splitter(s):
    #handles hyphens
    #e.g. pressure-treated to pressure treated, but keeps the the digits in place
    #e.g. sink strainer 3-1/2 to sink strainer 3-1/2
    try:
        s=re.sub(r"([a-zA-Z]+)[/\-]([a-zA-Z]+)", r"\1 \2",s)
        return s
    except:
        raise TypeError(s, 'is not a string')
        
def digit_comma_merger(s):
    #'2,000' to 2000
    try:
        s=re.sub(r"(?<=\d),(?=000)", r"",s)
        return s
    except:
        raise TypeError(s, 'is not a string')

def digit_letter_splitter(s):
    # 3x4x5 to 3 x 4 x 5
    try:
        s=re.sub(r"(\d+)[\.\-]*([a-zA-Z]+)", r"\1 \2",s)
        return s
    except:
        raise TypeError(s, 'is not a string')
    
def string_digit_mapper(s):
    #HANDLES ONLY TWO DIGIT NUMBERS (not 'three hundred ten')
    #produces gibberish otherwise
    #twenty three to 23
    #fourteen to 14
    numbers={'zero': '0','one': '1','two': '2', 'three': '3','four': '4',
             'five': '5','six': '6','seven': '7','eight': '8','nine': '9',
             'ten': '10', 'eleven': '11', 'twelve': '12', 'thirteen': '13',
             'fourteen': '14','fifteen': '15','sixteen': '16','seventeen': '17',
             'eighteen': '18', 'nineteen': '19','twenty': '20','thirty': '30',
             'forty': '40', 'fifty': '50','sixty': '60', 'seventy': '70',
              'eighty': '80','ninety': '90'}
    stop_numbers={'hundred':'100','thousand':'1000','million':'1000000'}
    
    r=s.split(' ')
    for ind, word in enumerate(r):
        if word in numbers:
            try:
                next_word=r[ind+1]
                if next_word in numbers and next_word not in stop_numbers:
                    try:
                        next_next_word=r[ind+2]
                        if next_next_word in numbers or next_next_word in stop_numbers:
                            return s
                        s=s.replace(word+' ','')
                        s=s.replace(word,'')
                        s=s.replace(next_word,numbers[word][0]+numbers[next_word])
                        return s

                    except:
                        s=s.replace(word+' ','')
                        s=s.replace(word,'')
                        s=s.replace(next_word,numbers[word][0]+numbers[next_word])
                        return s
                

            except:
                s=s.replace(word,numbers[word])
                return s
    return s


def lemmatiser(tokeniser=nltk.tokenize.TreebankWordTokenizer(),lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    
    return lambda x: " ".join(lemmatizer.lemmatize(token) for token in tokeniser.tokenize(x) if token not in stop_words)

def stemmer(tokeniser=nltk.tokenize.TreebankWordTokenizer(),lemmatizer=nltk.stem.SnowballStemmer('english')):
    #WARNING - LEMMATISER SHOULD BE THEORETICALLY BETTER

    return lambda x: " ".join(stemmer.stem(token) for token in tokeniser.tokenize(x) if token not in stop_words)

def strip_punctuation(tokeniser=nltk.tokenize.TreebankWordTokenizer()):
    #remove all punctuation (needed for product description) except for dots
    return lambda x: x.translate({ord(c): None for c in string.punctuation if c!='.'})
    #return lambda x: ' '.join(token for token in tokeniser.tokenize(x) if token not in stop_words)

def preprocessor():
        #THE ORDER IS FROM BOTTOM TO TOP (i.e. starting with x.lower())
    return lambda x:unit_converter(\
            string_digit_mapper(\
            digit_comma_merger(\
            digit_letter_splitter(\
            letter_splitter(\
            uppercase_splitter(\
            x.lower()
                                      )
                                          )
                                               )
                                                     )
                                                          )
                                                              )
def spell_checker():
        return lambda x:spelling_checker_dict[x] if x in spelling_checker_dict else x                                             

tokeniser = nltk.tokenize.TreebankWordTokenizer()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stemmer=nltk.stem.SnowballStemmer('english')



In [140]:
a=timeit.default_timer()
#don't know why two spell checkers - just to be safe I guess?
df_all['search_term']=df_all['search_term'].apply(spell_checker())
df_all['search_term']=df_all['search_term'].apply(preprocessor())
df_all['search_term']=df_all['search_term'].apply(spell_checker())
df_all['search_term']=df_all['search_term'].apply(lemmatiser())
b=timeit.default_timer()
print('preprocessing search term took {}s'.format(b-a))

a=timeit.default_timer()
df_all['product_title'] = df_all['product_title'].apply(preprocessor())
df_all['product_title'] = df_all['product_title'].apply(lemmatiser())
b=timeit.default_timer()
print('preprocessing product title took {}s'.format(b-a))

a=timeit.default_timer()
df_all['product_description'] = df_all['product_description'].apply(preprocessor())
df_all['product_description'] = df_all['product_description'].apply(lemmatiser())
b=timeit.default_timer()
print('preprocessing product description took {}s'.format(b-a))

preprocessing search term took 45.55640886700712s
preprocessing product title took 79.37594381900271s
preprocessing product description took 542.9778857789934s


In [142]:
#look for colour attributes
a=timeit.default_timer()

attr_names=df_attr.name.unique().tolist()
colour_names=[] #a list of unique colour attribute names
for name in attr_names:
    if type(name)==str:
        if 'Color' in name:
            colour_names.append(name)
      
            
df_colour = df_attr[df_attr.name.isin(colour_names)][["product_uid", "value"]]
df_colour.columns.values[1]='product_colour'

df_aggr_colour=df_colour.groupby(['product_uid'],as_index=False).aggregate(lambda x: ' '.join(list(set(x))))
#print(df_aggr_colour['product_uid'])
df_aggr_colour['product_colour']=df_aggr_colour['product_colour'].apply(lambda x: ' '.join(list(set(x.split()))))
df_aggr_colour['product_colour']=df_aggr_colour['product_colour'].apply(lambda x: unit_converter(x.lower()))
df_aggr_colour['product_colour']=df_aggr_colour['product_colour'].apply(lemmatiser())

b=timeit.default_timer()
print('preprocessing colour took {}s'.format(b-a))


a=timeit.default_timer()

df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]]
df_brand.columns.values[1]='brand' #renaming
df_brand['brand']= df_brand['brand'].apply(lemmatiser())

b=timeit.default_timer()
print('preprocessing brand took {}s'.format(b-a))


a=timeit.default_timer()

df_product_weight=df_attr[df_attr.name=='Product Weight (lb.)'][["product_uid", "value"]]
df_product_weight.columns.values[1]='product_weight'
df_product_weight['product_weight']=df_product_weight['product_weight'].apply(lambda x: unit_converter(x.lower()))
df_product_weight['product_weight']=df_product_weight['product_weight'].apply(lemmatiser())

df_product_depth=df_attr[df_attr.name=='Product Depth (in.)'][["product_uid", "value"]]
df_product_depth.columns.values[1]='product_depth'

df_product_depth['product_depth']=df_product_depth['product_depth'].apply(lambda x: unit_converter(x.lower()))
df_product_depth['product_depth']=df_product_depth['product_depth'].apply(lemmatiser())

df_product_height=df_attr[df_attr.name=='Product Height (in.)'][["product_uid", "value"]]
df_product_height.columns.values[1]='product_height'
df_product_height['product_height']=df_product_height['product_height'].apply(lambda x: unit_converter(x.lower()))
df_product_height['product_height']=df_product_height['product_height'].apply(lemmatiser())


df_product_width=df_attr[df_attr.name=='Product Width (in.)'][["product_uid", "value"]]
df_product_width.columns.values[1]='product_width'
df_product_width['product_width']=df_product_width['product_width'].apply(lambda x: unit_converter(x.lower()))
df_product_width['product_width']=df_product_width['product_width'].apply(lemmatiser())

b=timeit.default_timer()

print('preprocessing size took {}s'.format(b-a))

a=timeit.default_timer()
#ONLY FIRST 5 BULLET POINTS 
df_bullets_01=df_attr[df_attr.name=='Bullet01'][['product_uid','value']]
df_bullets_01.columns.values[1]='Bullet01'
df_bullets_01['Bullet01']=df_bullets_01['Bullet01'].apply(lambda x: unit_converter(x.lower()))
df_bullets_01['Bullet01']=df_bullets_01['Bullet01'].apply(lemmatiser())

df_bullets_02=df_attr[df_attr.name=='Bullet02'][['product_uid','value']]
df_bullets_02.columns.values[1]='Bullet02'
df_bullets_02['Bullet02']=df_bullets_02['Bullet02'].apply(lambda x: unit_converter(x.lower()))
df_bullets_02['Bullet02']=df_bullets_02['Bullet02'].apply(lemmatiser())

df_bullets_03=df_attr[df_attr.name=='Bullet03'][['product_uid','value']]
df_bullets_03.columns.values[1]='Bullet03'
df_bullets_03['Bullet03']=df_bullets_03['Bullet03'].apply(lambda x: unit_converter(x.lower()))
df_bullets_03['Bullet03']=df_bullets_03['Bullet03'].apply(lemmatiser())

df_bullets_04=df_attr[df_attr.name=='Bullet04'][['product_uid','value']]
df_bullets_04.columns.values[1]='Bullet04'
df_bullets_04['Bullet04']=df_bullets_04['Bullet04'].apply(lambda x: unit_converter(x.lower()))
df_bullets_04['Bullet04']=df_bullets_04['Bullet04'].apply(lemmatiser())

df_bullets_05=df_attr[df_attr.name=='Bullet05'][['product_uid','value']]
df_bullets_05.columns.values[1]='Bullet05'
df_bullets_05['Bullet05']=df_bullets_05['Bullet05'].apply(lambda x: unit_converter(x.lower()))
df_bullets_05['Bullet05']=df_bullets_05['Bullet05'].apply(lemmatiser())
b=timeit.default_timer()

print('preprocessing bulletpoints took {}s'.format(b-a))



preprocessing colour took 11.472802396980114s
preprocessing brand took 6.413068733003456s
preprocessing size took 27.16619769501267s
preprocessing bulletpoints took 97.45241981599247s


In [146]:
df_try = pd.merge(df_all, df_aggr_colour, how='left', on='product_uid')
df_try = pd.merge(df_try, df_brand, how='left', on='product_uid')
df_try = pd.merge(df_try, df_product_weight, how='left', on='product_uid')
df_try = pd.merge(df_try, df_product_depth, how='left', on='product_uid')
df_try = pd.merge(df_try, df_product_height, how='left', on='product_uid')
df_try = pd.merge(df_try, df_product_width, how='left', on='product_uid')
df_try = pd.merge(df_try, df_bullets_01, how='left', on='product_uid')
df_try = pd.merge(df_try, df_bullets_02, how='left', on='product_uid')
df_try = pd.merge(df_try, df_bullets_03, how='left', on='product_uid')
df_try = pd.merge(df_try, df_bullets_04, how='left', on='product_uid')
df_try = pd.merge(df_try, df_bullets_05, how='left', on='product_uid')

In [2]:
#df_all=df_try
df_all=pd.read_csv('raw.csv')
df_all=df_all[df_all.search_term.notnull()]

In [3]:
from difflib import SequenceMatcher
import Levenshtein
import distance

def sequence_match_score(s1,s2):
    #Ratcliff-Obershelp algorithm score
    seq=SequenceMatcher(None,s1,s2)
    
    return seq.ratio()

def levenshtein_dist(s1,s2):
    #normalized Levenshtein distance
    
    return Levenshtein.ratio(s1,s2)

def Jaccard_dist(s1,s2):
    #normalized Jaccardian distance
    set1, set2= set(s1.split()),set(s2.split())
    
    return float(len(set1 & set2)) / len(set1 | set2)

def full_string_match_count(s1, s2):
    
    #count the number of times s1 appears in s2
    count = 0
    index = 0
    while index < len(s2):
        index = s2.find(s1, index)
        if index == -1:
            #no match
            return count
        else:
            count += 1
            index += len(s1)
    return count

def common_words_match_count(s1, s2):
    
    #count the number of common words between s1 and s2
    return sum([int(s2.find(word)>=0) for word in s1.split()])

test_string=['hello world','hello world hello. My name is hello world']
print(full_string_match_count(test_string[0],test_string[1]))

print(common_words_match_count(test_string[0],test_string[1]))


ModuleNotFoundError: No module named 'Levenshtein'

In [ ]:
df_all['num_words_query'] = df_all['search_term'].map(lambda x:len(x.split()))
df_all['num_words_title'] = df_all['product_title'].map(lambda x:len(x.split()))
df_all['num_words_brand'] = df_all['brand'].map(lambda x:len(x.split()) if type(x)==str else 1)
df_all['num_words_colour'] = df_all['product_colour'].map(lambda x:len(x.split()) if type(x)==str else 1)
df_all['num_words_weight'] = df_all['product_weight'].map(lambda x:len(x.split()) if type(x)==str else 1)
df_all['num_words_height'] = df_all['product_height'].map(lambda x:len(x.split()) if type(x)==str else 1)
df_all['num_words_depth'] = df_all['product_depth'].map(lambda x:len(x.split()) if type(x)==str else 1 )
df_all['num_words_width'] = df_all['product_width'].map(lambda x:len(x.split()) if type(x)==str else 1 )



#SEARCH TERM AND PRODUCT TITLE SECTION
a=timeit.default_timer()

df_query_and_title=df_all['search_term'] + '||' + df_all['product_title']

df_all['query_in_title']=df_query_and_title.map(lambda x: full_string_match_count(
    x.split('||')[0],x.split('||')[1]))
df_all['common_words_query_and_title']=df_query_and_title.map(lambda x: common_words_match_count(
    x.split('||')[0],x.split('||')[1]))
df_all['query_last_word_in_title'] = df_query_and_title.map(lambda x:common_words_match_count(x.split('||')[0].split(" ")[-1],
                                                                                              x.split('||')[1]))

df_all['title_seq_match_score']=df_query_and_title.map(lambda x: sequence_match_score(x.split('||')[0],
                                                                                      x.split('||')[1]))

df_all['title_levenshtein_ratio']=df_query_and_title.map(lambda x: levenshtein_dist(x.split('||')[0],
                                                                                    x.split('||')[1]))

df_all['title_Jaccard_dist_norm']=df_query_and_title.map(lambda x: Jaccard_dist(x.split('||')[0],
                                                                                x.split('||')[1]))
df_all['ratio_title'] = df_all['common_words_query_and_title']/df_all['num_words_query']

del df_query_and_title
b=timeit.default_timer()

print('product title feature engineering took {}s'.format(b-a))

#SEARCH TERM AND DESCRIPTION
a=timeit.default_timer()

df_query_and_desc = df_all['search_term'] + '||' + df_all['product_description']
df_all['query_in_description'] = df_query_and_desc.map(lambda x: full_string_match_count(x.split('||')[0],
                                                                                         x.split('||')[1]))

df_all['common_words_query_and_desc']= df_query_and_desc.map(lambda x: common_words_match_count(
    x.split('||')[0],x.split('||')[1]))
df_all['query_last_word_in_desc'] = df_query_and_desc.map(lambda x:common_words_match_count(x.split('||')[0].split(" ")[-1],
                                                                                            x.split('||')[1]))

df_all['desc_sequence_match_score'] = df_query_and_desc.map(lambda x: sequence_match_score(x.split('||')[0],
                                                                                      x.split('||')[1]))
df_all['desc_levenshtein_ratio'] = df_query_and_desc.map(lambda x: levenshtein_dist(x.split('||')[0],
                                                                                x.split('||')[1]))
df_all['desc_Jaccard_dist_norm'] = df_query_and_desc.map(lambda x: Jaccard_dist(x.split('||')[0],
                                                                                x.split('||')[1]))

df_all['ratio_description'] = df_all['common_words_query_and_desc'] / df_all['num_words_query']

del df_query_and_desc 
b=timeit.default_timer()

print('product desc feature engineering took {}s'.format(b-a))

#SEARCH TERM AND BRAND
a=timeit.default_timer()

df_query_and_brand = df_all['search_term'] + '||' + df_all['brand']
df_query_and_brand.dropna(inplace=True)
df_all['query_in_brand'] = df_query_and_brand.map(lambda x: full_string_match_count(x.split('||')[0],
                                                                                    x.split('||')[1]))

df_all['common_words_query_and_brand']= df_query_and_brand.map(lambda x: common_words_match_count(x.split('||')[0],
    x.split('||')[1]))
#last word in brand doesn't make sense

df_all['brand_sequence_match_score'] = df_query_and_brand.map(lambda x: sequence_match_score(x.split('||')[0],
                                                                                      x.split('||')[1]))
df_all['brand_levenshtein_ratio'] = df_query_and_brand.map(lambda x: levenshtein_dist(x.split('||')[0],
                                                                                x.split('||')[1]))
df_all['brand_Jaccard_dist_norm'] = df_query_and_brand.map(lambda x: Jaccard_dist(x.split('||')[0],
                                                                                x.split('||')[1]))

df_all['ratio_brand'] = df_all['common_words_query_and_brand'] / df_all['num_words_query']

del df_query_and_brand
b=timeit.default_timer()

print('brand feature engineering took {}s'.format(b-a))

#SEARCH TERM AND COLOUR

a=timeit.default_timer()

df_query_and_colour = df_all['search_term']+'||'+df_all['product_colour']
df_query_and_colour.dropna(inplace=True) #dropna cause brands do not exist for every product uid and hence a search term
df_all['query_in_colour'] = df_query_and_colour.map(lambda x: full_string_match_count(x.split('||')[0],
                                                                                    x.split('||')[1]))

df_all['common_words_query_and_colour']= df_query_and_colour.map(lambda x: common_words_match_count(x.split('||')[0],
    x.split('||')[1]))
#last word in colour doesn't make sense
df_all['query_first_word_in_colour'] = df_query_and_colour.map(lambda x:common_words_match_count(x.split('||')[0].split(" ")[0],
                                                                                            x.split('||')[1]))

df_all['colour_sequence_match_score'] = df_query_and_colour.map(lambda x: sequence_match_score(x.split('||')[0],
                                                                                      x.split('||')[1]))
df_all['colour_levenshtein_ratio'] = df_query_and_colour.map(lambda x: levenshtein_dist(x.split('||')[0],
                                                                                x.split('||')[1]))
df_all['colour_Jaccard_dist_norm'] = df_query_and_colour.map(lambda x: Jaccard_dist(x.split('||')[0],
                                                                                x.split('||')[1]))

df_all['ratio_colour'] = df_all['common_words_query_and_colour'] / df_all['num_words_query']

del df_query_and_colour


b=timeit.default_timer()

print('colour feature engineering took {}s'.format(b-a))


#SEARCH TERM AND SIZE 

a=timeit.default_timer()
df_query_and_size = df_all['search_term'] + '||' + df_all['product_weight'] + ' ' +\
df_all['product_height'] + ' ' + df_all['product_depth'] + ' ' + df_all['product_width']

df_query_and_size.dropna(inplace=True) 

#query in size doesn't make sense
df_all['common_words_query_and_size']= df_query_and_size.map(lambda x: common_words_match_count(x.split('||')[0],
    x.split('||')[1]))

#first word in size doesn't make sense
df_all['query_last_word_in_size'] = df_query_and_size.map(lambda x:common_words_match_count(x.split('||')[0].split(" ")[-1],
                                                                                            x.split('||')[1]))

df_all['size_sequence_match_score'] = df_query_and_size.map(lambda x: sequence_match_score(x.split('||')[0],
                                                                                      x.split('||')[1]))
df_all['size_levenshtein_ratio'] = df_query_and_size.map(lambda x: levenshtein_dist(x.split('||')[0],
                                                                                x.split('||')[1]))
df_all['size_Jaccard_dist_norm'] = df_query_and_size.map(lambda x: Jaccard_dist(x.split('||')[0],
                                                                                x.split('||')[1]))

df_all['ratio_size'] = df_all['common_words_query_and_size'] / df_all['num_words_query']

del df_query_and_size

b=timeit.default_timer()

print('size feature engineering took {}s'.format(b-a))

#SEARCH TERM AND BULLETPOINTS
a=timeit.default_timer()

df_query_and_bullets = df_all['search_term'] + '||'  + df_all['Bullet01'] + df_all['Bullet02'] + ' ' +\
df_all['Bullet03'] + ' ' + df_all['Bullet04'] + ' ' + df_all['Bullet05']

df_query_and_bullets.dropna(inplace=True) 

df_all['query_in_bullets'] = df_query_and_bullets.map(lambda x: full_string_match_count(x.split('||')[0],
                                                                                    x.split('||')[1]))

df_all['common_words_query_and_bullets']= df_query_and_bullets.map(lambda x: common_words_match_count(x.split('||')[0],
    x.split('||')[1]))

#last word in bulletpoints doesn't make sense
df_all['bullets_sequence_match_score'] = df_query_and_bullets.map(lambda x: sequence_match_score(x.split('||')[0],
                                                                                      x.split('||')[1]))
df_all['bullets_levenshtein_ratio'] = df_query_and_bullets.map(lambda x: levenshtein_dist(x.split('||')[0],
                                                                                x.split('||')[1]))
df_all['bullets_Jaccard_dist_norm'] = df_query_and_bullets.map(lambda x: Jaccard_dist(x.split('||')[0],
                                                                                x.split('||')[1]))

df_all['ratio_bullets'] = df_all['common_words_query_and_bullets'] / df_all['num_words_query']

del df_query_and_bullets

b=timeit.default_timer()

print('bulletpoints feature engineering took {}s'.format(b-a))



In [ ]:
df_all.drop('Unnamed: 0',axis=1,inplace=True)
df_all

In [ ]:
df_all.to_csv('ready.csv')